In [ ]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

In [6]:
data = pd.read_csv('data/tmdb_5000_movies.csv')

In [7]:
data 


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures"", ""

In [8]:
df =data
df['genres']

0       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
1       [{"id": 12, "name": "Adventure"}, {"id": 14, "...
2       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
3       [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4       [{"id": 28, "name": "Action"}, {"id": 12, "nam...
                              ...                        
4798    [{"id": 28, "name": "Action"}, {"id": 80, "nam...
4799    [{"id": 35, "name": "Comedy"}, {"id": 10749, "...
4800    [{"id": 35, "name": "Comedy"}, {"id": 18, "nam...
4801                                                   []
4802                  [{"id": 99, "name": "Documentary"}]
Name: genres, Length: 4803, dtype: object

In [9]:
import ast

df['genres'] = df['genres'].map(lambda x: ast.literal_eval(x))

In [11]:
df['genres'][0][1]['name']

'Adventure'

In [12]:

for i in range(len(df['genres'])):
    for j in range(len(df['genres'][i])):
        df['genres'][i][j]=df['genres'][i][j]['name']

In [13]:
df['genres']

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4798                        [Action, Crime, Thriller]
4799                                [Comedy, Romance]
4800               [Comedy, Drama, Romance, TV Movie]
4801                                               []
4802                                    [Documentary]
Name: genres, Length: 4803, dtype: object

In [14]:
df['keywords'] = df['keywords'].map(lambda x: ast.literal_eval(x))

In [15]:

for i in range(len(df['keywords'])):
    for j in range(len(df['keywords'][i])):
        df['keywords'][i][j]=df['keywords'][i][j]['name']

In [16]:
df['keywords']

0       [culture clash, future, space war, space colon...
1       [ocean, drug abuse, exotic island, east india ...
2       [spy, based on novel, secret agent, sequel, mi...
3       [dc comics, crime fighter, terrorist, secret i...
4       [based on novel, mars, medallion, space travel...
                              ...                        
4798    [united states–mexico barrier, legs, arms, pap...
4799                                                   []
4800    [date, love at first sight, narration, investi...
4801                                                   []
4802            [obsession, camcorder, crush, dream girl]
Name: keywords, Length: 4803, dtype: object

In [17]:
combined_list = [df['keywords'],df['genres']]
print(combined_list)

[0       [culture clash, future, space war, space colon...
1       [ocean, drug abuse, exotic island, east india ...
2       [spy, based on novel, secret agent, sequel, mi...
3       [dc comics, crime fighter, terrorist, secret i...
4       [based on novel, mars, medallion, space travel...
                              ...                        
4798    [united states–mexico barrier, legs, arms, pap...
4799                                                   []
4800    [date, love at first sight, narration, investi...
4801                                                   []
4802            [obsession, camcorder, crush, dream girl]
Name: keywords, Length: 4803, dtype: object, 0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...               

In [18]:
combined = []
for i in range(len(df['keywords'])):
    combined.append([df['keywords'][i], df['genres'][i]])

print(combined)
combined_df = pd.DataFrame(combined, columns = ['keywords', 'genres'])
combined

[[['culture clash', 'future', 'space war', 'space colony', 'society', 'space travel', 'futuristic', 'romance', 'space', 'alien', 'tribe', 'alien planet', 'cgi', 'marine', 'soldier', 'battle', 'love affair', 'anti war', 'power relations', 'mind and soul', '3d'], ['Action', 'Adventure', 'Fantasy', 'Science Fiction']], [['ocean', 'drug abuse', 'exotic island', 'east india trading company', "love of one's life", 'traitor', 'shipwreck', 'strong woman', 'ship', 'alliance', 'calypso', 'afterlife', 'fighter', 'pirate', 'swashbuckler', 'aftercreditsstinger'], ['Adventure', 'Fantasy', 'Action']], [['spy', 'based on novel', 'secret agent', 'sequel', 'mi6', 'british secret service', 'united kingdom'], ['Action', 'Adventure', 'Crime']], [['dc comics', 'crime fighter', 'terrorist', 'secret identity', 'burglar', 'hostage drama', 'time bomb', 'gotham city', 'vigilante', 'cover-up', 'superhero', 'villainess', 'tragic hero', 'terrorism', 'destruction', 'catwoman', 'cat burglar', 'imax', 'flood', 'crimin

[[['culture clash',
   'future',
   'space war',
   'space colony',
   'society',
   'space travel',
   'futuristic',
   'romance',
   'space',
   'alien',
   'tribe',
   'alien planet',
   'cgi',
   'marine',
   'soldier',
   'battle',
   'love affair',
   'anti war',
   'power relations',
   'mind and soul',
   '3d'],
  ['Action', 'Adventure', 'Fantasy', 'Science Fiction']],
 [['ocean',
   'drug abuse',
   'exotic island',
   'east india trading company',
   "love of one's life",
   'traitor',
   'shipwreck',
   'strong woman',
   'ship',
   'alliance',
   'calypso',
   'afterlife',
   'fighter',
   'pirate',
   'swashbuckler',
   'aftercreditsstinger'],
  ['Adventure', 'Fantasy', 'Action']],
 [['spy',
   'based on novel',
   'secret agent',
   'sequel',
   'mi6',
   'british secret service',
   'united kingdom'],
  ['Action', 'Adventure', 'Crime']],
 [['dc comics',
   'crime fighter',
   'terrorist',
   'secret identity',
   'burglar',
   'hostage drama',
   'time bomb',
   'gotham 

In [104]:
combined_df.to_numpy()

array([[list(['culture clash', 'future', 'space war', 'space colony', 'society', 'space travel', 'futuristic', 'romance', 'space', 'alien', 'tribe', 'alien planet', 'cgi', 'marine', 'soldier', 'battle', 'love affair', 'anti war', 'power relations', 'mind and soul', '3d']),
        list(['Action', 'Adventure', 'Fantasy', 'Science Fiction'])],
       [list(['ocean', 'drug abuse', 'exotic island', 'east india trading company', "love of one's life", 'traitor', 'shipwreck', 'strong woman', 'ship', 'alliance', 'calypso', 'afterlife', 'fighter', 'pirate', 'swashbuckler', 'aftercreditsstinger']),
        list(['Adventure', 'Fantasy', 'Action'])],
       [list(['spy', 'based on novel', 'secret agent', 'sequel', 'mi6', 'british secret service', 'united kingdom']),
        list(['Action', 'Adventure', 'Crime'])],
       ...,
       [list(['date', 'love at first sight', 'narration', 'investigation', 'team', 'postal worker']),
        list(['Comedy', 'Drama', 'Romance', 'TV Movie'])],
       [list(

In [105]:
combined_df.shape

(4803, 2)

In [19]:
combined_df

,keywords,genres
0,"[culture clash, future, space war, space colon...","[Action, Adventure, Fantasy, Science Fiction]"
1,"[ocean, drug abuse, exotic island, east india ...","[Adventure, Fantasy, Action]"
2,"[spy, based on novel, secret agent, sequel, mi...","[Action, Adventure, Crime]"
3,"[dc comics, crime fighter, terrorist, secret i...","[Action, Crime, Drama, Thriller]"
4,"[based on novel, mars, medallion, space travel...","[Action, Adventure, Science Fiction]"
...,...,...
4798,"[united states–mexico barrier, legs, arms, pap...","[Action, Crime, Thriller]"
4799,[],"[Comedy, Romance]"
4800,"[date, love at first sight, narration, investi...","[Comedy, Drama, Romance, TV Movie]"
4801,[],[]


In [21]:
flat_string = combined_df.apply(lambda row: ' '.join([item for sublist in row for item in sublist]), axis=1)
print(flat_string)


0       culture clash future space war space colony so...
1       ocean drug abuse exotic island east india trad...
2       spy based on novel secret agent sequel mi6 bri...
3       dc comics crime fighter terrorist secret ident...
4       based on novel mars medallion space travel pri...
                              ...                        
4798    united states–mexico barrier legs arms paper k...
4799                                       Comedy Romance
4800    date love at first sight narration investigati...
4801                                                     
4802     obsession camcorder crush dream girl Documentary
Length: 4803, dtype: object


In [ ]:
df

In [27]:
inputs = flat_string
outputs = df['original_title']

In [31]:
outputs,inputs

(0                                         Avatar
 1       Pirates of the Caribbean: At World's End
 2                                        Spectre
 3                          The Dark Knight Rises
 4                                    John Carter
                           ...                   
 4798                                 El Mariachi
 4799                                   Newlyweds
 4800                   Signed, Sealed, Delivered
 4801                            Shanghai Calling
 4802                           My Date with Drew
 Name: original_title, Length: 4803, dtype: object,
 0       culture clash future space war space colony so...
 1       ocean drug abuse exotic island east india trad...
 2       spy based on novel secret agent sequel mi6 bri...
 3       dc comics crime fighter terrorist secret ident...
 4       based on novel mars medallion space travel pri...
                               ...                        
 4798    united states–mexico barrier legs a

In [44]:
tfidf = TfidfVectorizer()
X= tfidf.fit_transform(inputs)



In [45]:
# generate a mapping from movie title -> index (in df)
movie2idx = pd.Series(df.index, index=df['original_title'])
movie2idx

original_title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

In [57]:
X[0].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]], shape=(1, 7164))

In [56]:
X.shape

(4803, 7164)

In [49]:
def recommend(title):
  # get the row in the dataframe for this movie
  idx = movie2idx[title]
  if type(idx) is pd.Series:
    idx = idx.iloc[0]

  # calculate the pairwise similarities for this movie
  query = X[idx]
  scores = cosine_similarity(query, X)

  # currently the array is 1 x N, make it just a 1-D array
  scores = scores.flatten()

  # get the indexes of the highest scoring movies
  # get the first K recommendations
  # don't return itself!
  recommended_idx = (-scores).argsort()[1:6]

  # return the titles of the recommendations
  return outputs.iloc[recommended_idx]

In [50]:
print("Recommendations for 'Runaway Bride':")
print(recommend('Runaway Bride'))

Recommendations for 'Runaway Bride':
4115                    House of D
4045         Dancer, Texas Pop. 81
2290                  Just Married
2325    My Big Fat Greek Wedding 2
679                     Little Man
Name: original_title, dtype: object
